## Статистические тесты для анализа размеров мидий в городах

В качестве данных возьмем информацию о размерах раковины мидий, выращенных в Санкт-Петербурге и Магадане.

Необходимо узнать, есть ли различия в среднем размере мидии в зависимости от города-производителя. А также проверить данные на наличие корреляции.

Для выбора подходящего теста необходимо проверить данные на нормальность

In [2]:
import pandas as pd
import numpy as np

### 1. Загрузка данных


In [3]:
petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764, 0.0689]

In [4]:
# массивы имеют разную размерность. Для корректного выполнения тестов исключим лишнее значение большего массива
mussels_data = pd.DataFrame({'petersburg': petersburg, 'magadan': magadan[:-1]})
mussels_data

,petersburg,magadan
0,0.0974,0.1033
1,0.1352,0.0915
2,0.0817,0.0781
3,0.1016,0.0685
4,0.0968,0.0677
5,0.1064,0.0697
6,0.1050,0.0764


________________________________________________________________________________________________________________________
### 2. Проверка данных на нормальность


In [130]:
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (мы отвергаем H0)'

alpha = 0.05

#### 2.1. Тест Шапиро-Уилка

In [131]:
from scipy.stats import shapiro

def check_norm(column_name):
	_, p = shapiro(mussels_data[column_name])
	result = H0 if p > alpha else Ha
	print('%s: \np=%.3f. %s' % (column_name, p, result))
  
check_norm('petersburg')
check_norm('magadan')

petersburg: 
p=0.242. Данные распределены нормально
magadan: 
p=0.133. Данные распределены нормально


$p_{petersburg} > \alpha$  $(p_{petersburg} = 0.242, \alpha = 0.05)$

$p_{magadan} > \alpha$  $(p_{magadan} = 0.133, \alpha = 0.05)$

При выборе статистических тестов будем считать, что размеры мидий в обоих городах распределены нормально.

Однако тот факт, что мы не можем отвергнуть нулевую гипотезу, не гарантирует того, что данные распределены нормально. И значения p-value не велики. 

Заметим, что если бы мы не обрезали последнее значение массива magadan из-за разности в размерностях массивов, данные по Магадану не были бы распределены нормально. И мы бы использовали другие тесты.

In [132]:
_, p = shapiro(magadan)
result = H0 if p > alpha else Ha
print('magadan: \np=%.3f. %s' % (p, result))

magadan: 
p=0.036. Данные не распределены нормально (мы отвергаем H0)


#### 2.2. Тест Д’Агостино

Не может быть выполнен из-за слишком малого числа экспериментов

__________________________________________________________________________________________________________________________
### 3. Проверка наличия корреляции

Для проверки корреляции выберем тест Пирсона, т.к. данные распределены нормально.

Если бы использовали полный массив данных по Магадану, следовало бы использовать тест Спирмена.

In [133]:
H0 = 'Мы не можем отвергнуть нулевую гипотезу об отсутствии зависимости между переменными.'
Ha = 'Мы отвергаем нулевую гипотезу об отсутствии зависимости между переменными'

alpha = 0.05

In [134]:
from scipy.stats import pearsonr

corr, p = pearsonr(mussels_data['petersburg'], mussels_data['magadan'])

result = H0 if p > alpha else Ha
print('Коэффициент корреляции = %.3f, p = %.3f.\n%s' % (corr, p, result))


Коэффициент корреляции = 0.255, p = 0.581.
Мы не можем отвергнуть нулевую гипотезу об отсутствии зависимости между переменными.


$p > \alpha$ $(p = 0.581, \alpha = 0.05)$

Т.о. мы не смогли убедиться в наличии зависимости между размерами мидий в разных городах. 

Также об этом свидетельствует коэффициент корреляции. Он довольно низкий и соответствует силе связи **отсутствие связи или очень слабая связь**.

_________________________________________________________________________________________________________________
### 4. Проверка гипотезы о разнице между размерами мидий

In [10]:
import plotly.express as px
px.histogram(mussels_data, barmode='overlay', nbins=8)

Визуально кажется, что средние не равны. Проверим статистическими тестами.

In [135]:
H0 = 'Нет значимой разницы между средним размером раковины мидий в Петербурге и Магадане.'
Ha = 'Есть значимая разница между средним размером раковины мидий в Петербурге и Магадане.'

alpha = 0.05

Данные нормально распределены, группы происходят из разных совокупностей, объектов в выборке 7 (< 30). 

Значит, мы используем независимый T-тест.

In [136]:
from scipy.stats import ttest_ind

_, p = ttest_ind(mussels_data['petersburg'], mussels_data['magadan'], equal_var=True)
result = H0 if p > alpha else Ha

print('p = %.3f.\n%s' % (p, result))


p = 0.010.
Есть значимая разница между средним размером раковины мидий в Петербурге и Магадане.


$p < \alpha$ $(p = 0.01, \alpha = 0.05)$

При уровне значимости 0.05 мы приняли альтернативную гипотезу о том, что средний размер мидий в Магадане и Санкт-Петербурге отличается.

Отметим, что пример является учебным. И в реальности объединение данных не имеет смысла, т.к. эксперименты в разных городах никак не связаны между собой. 